In [3]:
import requests as rq
import os
from dotenv import load_dotenv
from tqdm.notebook import tqdm
from copy import deepcopy
from time import sleep
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)


load_dotenv()

True

In [2]:
token = os.getenv('access_token')
base_url = 'https://api.vk.com/method/'
v = "5.199"
need_info = 'first_name, last_name, deactivated, is_closed, about, activities, bdate, books, career, city, common_count, education, friend_status, is_friend, last_seen, personal, relation, schools, universities, sex, status'

In [3]:
my_friends = rq.get(
    base_url + 'friends.get',
    params={
        'access_token': token,
        'v': v
    }
).json()['response']['items']

In [4]:
my_friends[:5]

[1193834, 1762723, 2612800, 4510163, 37460188]

In [5]:
friends = []

In [6]:
for friend_id in tqdm(my_friends):
    user_info = rq.get(
        base_url + "users.get",
        params={
            "user_id": str(friend_id),
            "v": v,
            "access_token": token,
            "name_case": "nom",
            "fields": need_info
        }
    ).json()['response'][0]
    mutual_friends = rq.get(
        base_url + "friends.getMutual",
        params={
            'target_uid': str(friend_id),
            "access_token": token,
            "v": v
        }
    ).json().get('response', 'error')
    user_info['mutual_friends'] = mutual_friends
    friends.append(deepcopy(user_info))
    sleep(0.5)

  0%|          | 0/98 [00:00<?, ?it/s]

In [35]:
import pickle
with open('friends.pkl', 'wb') as f:
    pickle.dump(friends, f)

In [7]:
import pandas as pd

In [11]:
pd.set_option('display.max_columns', None)

In [32]:
df = pd.DataFrame(friends)
df.relation = df.relation.astype(str)

In [33]:
df.relation.value_counts()

relation
0.0    54
nan    25
4.0     5
1.0     4
6.0     3
7.0     3
3.0     2
2.0     1
5.0     1
Name: count, dtype: int64

In [36]:
relation = {
    '0.0': 'не указано',
    'nan': 'не указано',
    '1.0': 'не женат/не замужем',
    '2.0': 'есть друг/есть подруга',
    '3.0': 'помолвлен/помолвлена',
    '4.0': 'женат/замужем',
    '5.0': 'всё сложно',
    '6.0': 'в активном поиске',
    '7.0': 'в гражданском браке'
}

In [40]:
df.relation = df.relation.apply(lambda x: relation[x])

In [43]:
df.city = df.city.apply(lambda x: x.get('title') if pd.notna(x) else None)

In [54]:
df['political'] = df.personal.apply(lambda x: x.get('political') if pd.notna(x) else None)
df['alcohol'] = df.personal.apply(lambda x: x.get('alcohol') if pd.notna(x) else None)
df['life_main'] = df.personal.apply(lambda x: x.get('life_main') if pd.notna(x) else None)
df['people_main'] = df.personal.apply(lambda x: x.get('people_main') if pd.notna(x) else None)
df['religion'] = df.personal.apply(lambda x: x.get('religion') if pd.notna(x) else None)
df['smoking'] = df.personal.apply(lambda x: x.get('smoking') if pd.notna(x) else None)
df['inspired_by'] = df.personal.apply(lambda x: x.get('inspired_by') if pd.notna(x) else None)

In [58]:
df['name'] = df['first_name'] + ' ' + df['last_name']

In [55]:
df.head()

,id,bdate,city,is_friend,books,about,activities,status,last_seen,common_count,career,university,university_name,faculty,faculty_name,graduation,relation,personal,universities,schools,sex,friend_status,first_name,last_name,can_access_closed,is_closed,mutual_friends,education_form,education_status,relation_partner,deactivated,political,alcohol,life_main,people_main,religion,smoking,inspired_by
0,1193834,18.1,Тверь,1.0,,,,Необычные экскурсии и авторские туры по Твери:...,"{'platform': 4, 'time': 1732389375}",3.0,[],0.0,,0.0,,0.0,не указано,"{'langs': ['Русский'], 'langs_full': [{'id': 0...",[],[],2,3,Игорь,Марченкофф,True,False,"[54824441, 179527549, 385680044]",NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,Православие,NaN,None
1,1762723,18.1.1970,Тверь,1.0,"""Мастер и Маргарита""",,Преподавание,Every day another miracle...,"{'platform': 1, 'time': 1732385835}",5.0,[],841.0,ТвГУ,10928.0,Иностранных языков и международной коммуникацмм,1993.0,не указано,{'religion': 'Православие'},"[{'city': 141, 'education_form': 'Очное отделе...","[{'city': 175, 'class': '', 'class_id': 0, 'id...",1,3,Елена,Круглая,True,True,"[172826326, 187783283, 232523985, 263499805, 4...",Очное отделение,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Православие,NaN,None
2,2612800,22.11,Санкт-Петербург,1.0,NaN,NaN,NaN,"Я верю в силу смеха. И мне кажется, что вы лег...","{'platform': 2, 'time': 1698554337}",1.0,NaN,NaN,NaN,NaN,NaN,NaN,не указано,NaN,NaN,NaN,1,3,Мария,Остапова,True,False,[276078718],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None
3,4510163,18.11,Москва,1.0,,,https://vk.com/clubdarkprincess,"У Бога всё бывает вовремя! Особенно для тех, к...","{'platform': 2, 'time': 1732384466}",5.0,"[{'city_id': 1, 'from': 2024, 'group_id': 6546...",1187585.0,РТУ МИРЭА,0.0,,0.0,помолвлен/помолвлена,"{'alcohol': 0, 'inspired_by': '', 'life_main':...","[{'city': 1, 'education_status': 'Кандидат нау...",[],1,3,Ольга,Александровна,True,True,"[166700184, 168449968, 211630257, 233958378, 3...",NaN,Кандидат наук,NaN,NaN,NaN,0.0,1.0,2.0,None,1.0,
4,37460188,1.3,None,1.0,,Главное нововведение 2012 года!\nУРА!\nТеперь ...,,миник ???кк,"{'platform': 7, 'time': 1540123162}",0.0,"[{'city_id': 110, 'company': '!VIP!'}, {'city_...",0.0,,0.0,,0.0,не указано,"{'alcohol': 1, 'inspired_by': '', 'langs': ['Р...",[],"[{'city': 110, 'class': 'г', 'class_id': 4, 'i...",2,3,Василий,Уткин,True,False,[],NaN,NaN,NaN,NaN,3.0,1.0,2.0,2.0,Православие,1.0,


In [62]:
cols2drop = [
    'is_friend',
    'books',
    'about',
    'status',
    'activities',
    'last_seen',
    'career',
    'university',
    'faculty',
    'faculty_name',
    'personal',
    'friend_status',
    'can_access_closed',
    'education_form',
    'deactivated',
    'inspired_by',
    'first_name',
    'last_name'
]

In [87]:
df_new = df.drop(columns=cols2drop)
df_new

,id,bdate,city,common_count,university_name,graduation,relation,universities,schools,sex,is_closed,mutual_friends,education_status,relation_partner,political,alcohol,life_main,people_main,religion,smoking,name
0,1193834,18.1,Тверь,3.0,,0.0,не указано,[],[],2,False,"[54824441, 179527549, 385680044]",NaN,NaN,6.0,NaN,NaN,NaN,Православие,NaN,Игорь Марченкофф
1,1762723,18.1.1970,Тверь,5.0,ТвГУ,1993.0,не указано,"[{'city': 141, 'education_form': 'Очное отделе...","[{'city': 175, 'class': '', 'class_id': 0, 'id...",1,True,"[172826326, 187783283, 232523985, 263499805, 4...",NaN,NaN,NaN,NaN,NaN,NaN,Православие,NaN,Елена Круглая
2,2612800,22.11,Санкт-Петербург,1.0,NaN,NaN,не указано,NaN,NaN,1,False,[276078718],NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,Мария Остапова
3,4510163,18.11,Москва,5.0,РТУ МИРЭА,0.0,помолвлен/помолвлена,"[{'city': 1, 'education_status': 'Кандидат нау...",[],1,True,"[166700184, 168449968, 211630257, 233958378, 3...",Кандидат наук,NaN,NaN,0.0,1.0,2.0,None,1.0,Ольга Александровна
4,37460188,1.3,None,0.0,,0.0,не указано,[],"[{'city': 110, 'class': 'г', 'class_id': 4, 'i...",2,False,[],NaN,NaN,3.0,1.0,2.0,2.0,Православие,1.0,Василий Уткин
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,466114898,30.10,Тверь,15.0,,0.0,не указано,[],[],2,False,"[1762723, 130865054, 135027781, 151925195, 158...",NaN,NaN,NaN,0.0,0.0,0.0,None,0.0,Сергей Валиев
94,505679418,3.1.1998,Тверь,0.0,ТвГУ,0.0,не указано,"[{'chair': 2061924, 'chair_name': 'Юриспруденц...","[{'city': 141, 'id': '93755', 'name': 'Тверско...",2,True,[],NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,Дмитрий Новиков
95,554605837,27.11,None,1.0,ТвГУ,0.0,не указано,"[{'city': 141, 'id': 841, 'name': 'ТвГУ'}]",[],2,False,[466114898],NaN,NaN,NaN,0.0,0.0,0.0,None,0.0,Алексей Гешко
96,561048343,12.12,Москва,0.0,,0.0,не указано,[],[],1,False,[],NaN,NaN,NaN,0.0,0.0,0.0,None,0.0,Аделина Исенцова


In [91]:
df_new.universities = df_new.universities.apply(lambda x: [] if x is np.nan else x)
df_new.schools = df_new.schools.apply(lambda x: [] if x is np.nan else x)

In [96]:
df_new.graduation = df_new.graduation.apply(lambda x: -1 if pd.isna(x) or x == 0 else x)

In [99]:
df_new.graduation = df_new.graduation.astype(int)

In [104]:
df_new['deactivated'] = df['deactivated']

In [109]:
df_new.common_count = df_new.common_count.fillna(0).astype(int)

In [115]:
df_new.political = df_new.political.fillna(-1).astype(int)
df_new.alcohol = df_new.alcohol.fillna(-1).astype(int)
df_new.life_main = df_new.life_main.fillna(-1).astype(int)
df_new.smoking = df_new.smoking.fillna(-1).astype(int)

In [118]:
df_new.religion = df_new.religion.fillna('Не указано')

In [145]:
political = {
    1: 'коммунистические',
    2: 'социалистические',
    3: 'умеренные',
    4: 'либеральные',
    5: 'консервативные',
    6: 'монархические',
    7: 'ультраконсервативные',
    8: 'индифферентные',
    9: 'либертарианские',
    0: None,
    -1: None
}

people_main = {
    1: 'ум и креативность',
    2: 'доброта и честность',
    3: 'красота и здоровье',
    4: 'власть и богатство',
    5: 'смелость и упорство',
    6: 'юмор и жизнелюбие',
    0: None,
    -1: None
}

life_main = {
    1: 'семья и дети',
    2: 'карьера и деньги',
    3: 'развлечения и отдых',
    4: 'наука и исследования',
    5: 'совершенствование мира',
    6: 'саморазвитие',
    7: 'красота и искусство',
    8: 'слава и влияние',
    0: None,
    -1: None
}

smoking = {
    1: 'резко негативное',
    2: 'негативное',
    3: 'компромиссное',
    4: 'нейтральное',
    5: 'положительное',
    0: None,
    -1: None
}

alcohol = {
    1: 'резко негативное',
    2: 'негативное',
    3: 'компромиссное',
    4: 'нейтральное',
    5: 'положительное',
    0: None,
    -1: None
}

In [146]:
df_new.political.value_counts()

political
умеренные           2
индифферентные      2
монархические       1
коммунистические    1
либертарианские     1
социалистические    1
консервативные      1
Name: count, dtype: int64

In [144]:
df_new.people_main = df_new.people_main.fillna(-1).astype(int)

In [147]:
#df_new.political = df_new.political.apply(lambda x: political[x])
#df_new.political = df_new.political.apply(lambda x: political[x])
df_new.people_main = df_new.people_main.apply(lambda x: people_main[x])
df_new.life_main = df_new.life_main.apply(lambda x: life_main[x])
df_new.smoking = df_new.smoking.apply(lambda x: smoking[x])
df_new.alcohol = df_new.alcohol.apply(lambda x: alcohol[x])

In [148]:
df_new.head()

,id,bdate,city,common_count,university_name,graduation,relation,universities,schools,sex,is_closed,mutual_friends,education_status,relation_partner,political,alcohol,life_main,people_main,religion,smoking,name,deactivated
0,1193834,18.1,Тверь,3,,-1,не указано,[],[],2,False,"[54824441, 179527549, 385680044]",NaN,NaN,монархические,None,None,None,Православие,None,Игорь Марченкофф,NaN
1,1762723,18.1.1970,Тверь,5,ТвГУ,1993,не указано,"[{'city': 141, 'education_form': 'Очное отделе...","[{'city': 175, 'class': '', 'class_id': 0, 'id...",1,True,"[172826326, 187783283, 232523985, 263499805, 4...",NaN,NaN,None,None,None,None,Православие,None,Елена Круглая,NaN
2,2612800,22.11,Санкт-Петербург,1,NaN,-1,не указано,[],[],1,False,[276078718],NaN,NaN,None,None,None,None,Не указано,None,Мария Остапова,NaN
3,4510163,18.11,Москва,5,РТУ МИРЭА,-1,помолвлен/помолвлена,"[{'city': 1, 'education_status': 'Кандидат нау...",[],1,True,"[166700184, 168449968, 211630257, 233958378, 3...",Кандидат наук,NaN,None,None,семья и дети,доброта и честность,Не указано,резко негативное,Ольга Александровна,NaN
4,37460188,1.3,None,0,,-1,не указано,[],"[{'city': 110, 'class': 'г', 'class_id': 4, 'i...",2,False,[],NaN,NaN,умеренные,резко негативное,карьера и деньги,доброта и честность,Православие,резко негативное,Василий Уткин,NaN


In [160]:
df_new[df_new.deactivated.notna()]

,id,bdate,city,common_count,university_name,graduation,relation,universities,schools,sex,is_closed,mutual_friends,education_status,relation_partner,political,alcohol,life_main,people_main,religion,smoking,name,deactivated
13,98940718,12.3.1966,Зеленокумск,0,NaN,-1,не указано,[],[],1,False,error,NaN,NaN,None,None,None,None,Не указано,None,Зерифа Остапова,banned
14,109255643,13.12,Казань,0,NaN,-1,не указано,[],[],2,False,error,NaN,NaN,None,None,None,None,Не указано,None,Максим Валиев,banned
31,170304526,1.4.2000,Los Angeles,0,NaN,-1,не указано,[],[],2,False,error,NaN,NaN,None,None,None,None,Не указано,None,Александр Беляев,banned
91,450371875,NaN,College Station,0,NaN,-1,не указано,[],[],1,False,error,NaN,NaN,None,None,None,None,Не указано,None,General Invoker,banned


In [163]:
df_new = df_new[df_new.deactivated.isna()].reset_index(drop=True)

In [164]:
df_new.to_parquet('friends.parquet', index=False)

In [165]:
df_new.head()

,id,bdate,city,common_count,university_name,graduation,relation,universities,schools,sex,is_closed,mutual_friends,education_status,relation_partner,political,alcohol,life_main,people_main,religion,smoking,name,deactivated
0,1193834,18.1,Тверь,3,,-1,не указано,[],[],2,False,"[54824441, 179527549, 385680044]",NaN,NaN,монархические,None,None,None,Православие,None,Игорь Марченкофф,NaN
1,1762723,18.1.1970,Тверь,5,ТвГУ,1993,не указано,"[{'city': 141, 'education_form': 'Очное отделе...","[{'city': 175, 'class': '', 'class_id': 0, 'id...",1,True,"[172826326, 187783283, 232523985, 263499805, 4...",NaN,NaN,None,None,None,None,Православие,None,Елена Круглая,NaN
2,2612800,22.11,Санкт-Петербург,1,NaN,-1,не указано,[],[],1,False,[276078718],NaN,NaN,None,None,None,None,Не указано,None,Мария Остапова,NaN
3,4510163,18.11,Москва,5,РТУ МИРЭА,-1,помолвлен/помолвлена,"[{'city': 1, 'education_status': 'Кандидат нау...",[],1,True,"[166700184, 168449968, 211630257, 233958378, 3...",Кандидат наук,NaN,None,None,семья и дети,доброта и честность,Не указано,резко негативное,Ольга Александровна,NaN
4,37460188,1.3,None,0,,-1,не указано,[],"[{'city': 110, 'class': 'г', 'class_id': 4, 'i...",2,False,[],NaN,NaN,умеренные,резко негативное,карьера и деньги,доброта и честность,Православие,резко негативное,Василий Уткин,NaN


In [168]:
df_new[df_new.universities.apply(lambda x: len(x) != 0) & df_new.university_name.isna()]

,id,bdate,city,common_count,university_name,graduation,relation,universities,schools,sex,is_closed,mutual_friends,education_status,relation_partner,political,alcohol,life_main,people_main,religion,smoking,name,deactivated


In [169]:
df_new.schools.apply(len).value_counts()

schools
0    63
1    30
2     1
Name: count, dtype: int64

In [170]:
df_new[df_new.schools.apply(lambda x: len(x) == 2)]

,id,bdate,city,common_count,university_name,graduation,relation,universities,schools,sex,is_closed,mutual_friends,education_status,relation_partner,political,alcohol,life_main,people_main,religion,smoking,name,deactivated
28,168449968,2.2.2002,Москва,14,РТУ МИРЭА,-1,в гражданском браке,"[{'chair': 2031563, 'chair_name': 'Программная...","[{'city': 95, 'class': 'б', 'class_id': 2, 'id...",2,False,"[4510163, 47050558, 166700184, 180806646, 1963...",Студент (бакалавр),"{'id': 241181127, 'first_name': 'Ольга', 'last...",None,None,None,None,Не указано,None,Денис Сироткин,NaN


In [4]:
df = pd.read_parquet('friends.parquet')

In [5]:
df

,id,bdate,city,common_count,university_name,graduation,relation,universities,schools,sex,is_closed,mutual_friends,education_status,relation_partner,political,alcohol,life_main,people_main,religion,smoking,name,deactivated
0,1193834,18.1,Тверь,3,,-1,не указано,[],[],2,False,"[54824441, 179527549, 385680044]",None,None,монархические,None,None,None,Православие,None,Игорь Марченкофф,None
1,1762723,18.1.1970,Тверь,5,ТвГУ,1993,не указано,"[{'chair': None, 'chair_name': None, 'city': 1...","[{'city': 175, 'class': '', 'class_id': 0.0, '...",1,True,"[172826326, 187783283, 232523985, 263499805, 4...",None,None,None,None,None,None,Православие,None,Елена Круглая,None
2,2612800,22.11,Санкт-Петербург,1,None,-1,не указано,[],[],1,False,[276078718],None,None,None,None,None,None,Не указано,None,Мария Остапова,None
3,4510163,18.11,Москва,5,РТУ МИРЭА,-1,помолвлен/помолвлена,"[{'chair': None, 'chair_name': None, 'city': 1...",[],1,True,"[166700184, 168449968, 211630257, 233958378, 3...",Кандидат наук,None,None,None,семья и дети,доброта и честность,Не указано,резко негативное,Ольга Александровна,None
4,37460188,1.3,None,0,,-1,не указано,[],"[{'city': 110, 'class': 'г', 'class_id': 4.0, ...",2,False,[],None,None,умеренные,резко негативное,карьера и деньги,доброта и честность,Православие,резко негативное,Василий Уткин,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,466114898,30.10,Тверь,15,,-1,не указано,[],[],2,False,"[1762723, 130865054, 135027781, 151925195, 158...",None,None,None,None,None,None,Не указано,None,Сергей Валиев,None
90,505679418,3.1.1998,Тверь,0,ТвГУ,-1,не указано,"[{'chair': 2061924.0, 'chair_name': 'Юриспруде...","[{'city': 141, 'class': None, 'class_id': None...",2,True,[],None,None,None,None,None,None,Не указано,None,Дмитрий Новиков,None
91,554605837,27.11,None,1,ТвГУ,-1,не указано,"[{'chair': None, 'chair_name': None, 'city': 1...",[],2,False,[466114898],None,None,None,None,None,None,Не указано,None,Алексей Гешко,None
92,561048343,12.12,Москва,0,,-1,не указано,[],[],1,False,[],None,None,None,None,None,None,Не указано,None,Аделина Исенцова,None


In [7]:
df.relation = df.relation.apply(lambda x: None if x == 'не указано' else x)

In [10]:
df.religion = df.religion.apply(lambda x: None if x == 'Не указано' else x)

In [11]:
df

,id,bdate,city,common_count,university_name,graduation,relation,universities,schools,sex,is_closed,mutual_friends,education_status,relation_partner,political,alcohol,life_main,people_main,religion,smoking,name,deactivated
0,1193834,18.1,Тверь,3,,-1,None,[],[],2,False,"[54824441, 179527549, 385680044]",None,None,монархические,None,None,None,Православие,None,Игорь Марченкофф,None
1,1762723,18.1.1970,Тверь,5,ТвГУ,1993,None,"[{'chair': None, 'chair_name': None, 'city': 1...","[{'city': 175, 'class': '', 'class_id': 0.0, '...",1,True,"[172826326, 187783283, 232523985, 263499805, 4...",None,None,None,None,None,None,Православие,None,Елена Круглая,None
2,2612800,22.11,Санкт-Петербург,1,None,-1,None,[],[],1,False,[276078718],None,None,None,None,None,None,None,None,Мария Остапова,None
3,4510163,18.11,Москва,5,РТУ МИРЭА,-1,помолвлен/помолвлена,"[{'chair': None, 'chair_name': None, 'city': 1...",[],1,True,"[166700184, 168449968, 211630257, 233958378, 3...",Кандидат наук,None,None,None,семья и дети,доброта и честность,None,резко негативное,Ольга Александровна,None
4,37460188,1.3,None,0,,-1,None,[],"[{'city': 110, 'class': 'г', 'class_id': 4.0, ...",2,False,[],None,None,умеренные,резко негативное,карьера и деньги,доброта и честность,Православие,резко негативное,Василий Уткин,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,466114898,30.10,Тверь,15,,-1,None,[],[],2,False,"[1762723, 130865054, 135027781, 151925195, 158...",None,None,None,None,None,None,None,None,Сергей Валиев,None
90,505679418,3.1.1998,Тверь,0,ТвГУ,-1,None,"[{'chair': 2061924.0, 'chair_name': 'Юриспруде...","[{'city': 141, 'class': None, 'class_id': None...",2,True,[],None,None,None,None,None,None,None,None,Дмитрий Новиков,None
91,554605837,27.11,None,1,ТвГУ,-1,None,"[{'chair': None, 'chair_name': None, 'city': 1...",[],2,False,[466114898],None,None,None,None,None,None,None,None,Алексей Гешко,None
92,561048343,12.12,Москва,0,,-1,None,[],[],1,False,[],None,None,None,None,None,None,None,None,Аделина Исенцова,None


In [13]:
df.schools.value_counts()

schools
[{'city': 141, 'class': None, 'class_id': None, 'id': '436384', 'name': 'Многопрофильная гимназия №12 (бывш. шк. 12)', 'speciality': None, 'type': 1.0, 'type_str': 'Гимназия', 'year_from': None, 'year_graduated': None, 'year_to': None}]                                                                                                                                                                                                                     4
[]                                                                                                                                                                                                                                                                                                                                                                                                                                                               1
[{'city': 282, 'class': '', 'class_id': 0.0, 'id': '251814', 'name': 'Минский государствен

In [16]:
df['school_name'] = df.schools.apply(lambda x: x[0]['name'] if len(x) > 0 else None)

In [17]:
df.to_parquet('friends2.parquet', index=False)